In [1]:
!pip install fleet-python


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [20]:
# Load environment variables from .env file
import os
from pathlib import Path

# Load .env file if it exists
env_file = Path('.env')
if env_file.exists():
    with open(env_file) as f:
        for line in f:
            if line.strip() and not line.startswith('#'):
                key, value = line.strip().split('=', 1)
                os.environ[key] = value
    print("✅ Loaded .env file")
else:
    print("❌ .env file not found")

# Verify API key is loaded
api_key = os.environ.get('FLEET_API_KEY')
if api_key:
    print(f"✅ API key loaded: {api_key[:8]}...{api_key[-4:]}")
else:
    print("❌ FLEET_API_KEY not found in environment")

✅ Loaded .env file
✅ API key loaded: sk_WTvuk...wcwM


In [22]:
# Test import and basic functionality

import fleet as flt

print(f"✅ Fleet SDK imported successfully!")
print(f"📦 Version: {flt.__version__}")
print(f"🔧 Available modules: {[x for x in dir(flt) if not x.startswith('_')]}")


✅ Fleet SDK imported successfully!
📦 Version: 0.1.0
🔧 Available modules: ['APIEnvironment', 'FleetAPIClient', 'FleetAPIError', 'FleetConfig', 'FleetConfigurationError', 'FleetError', 'FleetManagerClient', 'FleetTimeoutError', 'HealthResponse', 'InstanceRequest', 'InstanceResponse', 'InstanceURLs', 'ManagerHealthResponse', 'ManagerURLs', 'TimestampResponse', 'client', 'config', 'env', 'exceptions', 'facets', 'get_config', 'manager_client']


In [ ]:
# List available environments

def list_available_environments():
    """List all available environments on the platform."""
    try:
        environments = flt.env.list_envs()
        print(f"🌍 Available environments ({len(environments)}):")
        print("-" * 50)
        
        for env in environments[:10]:  # Show first 10
            print(f"🔹 {env.env_key}")
            if env.description:
                print(f"   📝 {env.description}")
            if env.default_version:
                print(f"   📋 Default version: {env.default_version}")
            print()
            
        if len(environments) > 10:
            print(f"... and {len(environments) - 10} more environments")
            
        return environments
    except Exception as e:
        print(f"❌ Error listing environments: {e}")
        return []

# Run the function
environments = list_available_environments()


🌍 Available environments (4):
--------------------------------------------------
🔹 confluence
   📝 Confluence with Jira access
   📋 Default version: v1.2.5

🔹 dropbox
   📝 File storage and sharing platform with cloud sync capabilities
   📋 Default version: v1.3.2

🔹 fira
   📝 Jira, Jira Service Management, Jira Product Discovery, with Confluence access
   📋 Default version: v1.2.5

🔹 hubspot
   📝 CRM and marketing automation for growing businesses
   📋 Default version: v1.2.3



In [ ]:
# Create a new environment instance

def create_test_environment():
    """Create a new environment instance for testing."""
    try:
        print("🚀 Creating new environment instance...")
        
        # Create environment (this will actually create an instance)
        env = flt.env.make("fira")  # Using 'fira' as it's commonly available
        
        print(f"✅ Environment instance created successfully!")
        print(f"🆔 Instance ID: {env.instance_id}")
        print(f"🏷️  Environment Key: {env.env_key}")
        print(f"📍 Region: {env.region}")
        print(f"🔗 Status: Creating/Starting...")
        
        return env
        
    except Exception as e:
        print(f"❌ Error creating environment: {e}")
        import traceback
        traceback.print_exc()
        return None

# Create the environment
test_env = create_test_environment()


🚀 Creating new environment instance...
✅ Environment instance created successfully!
🆔 Instance ID: 5b9fc55c
❌ Error creating environment: 'RemoteEnvironment' object has no attribute 'env_key'


Traceback (most recent call last):
  File "/var/folders/gb/s4mkbx1j5x187xr7c5rldcrw0000gn/T/ipykernel_91425/4009314754.py", line 13, in create_test_environment
    print(f"🏷️  Environment Key: {env.env_key}")
AttributeError: 'RemoteEnvironment' object has no attribute 'env_key'


In [ ]:
# List all running instances to verify our environment is there

def list_running_instances():
    """List all running environment instances."""
    try:
        print("📋 Listing all running instances...")
        
        # Get all running instances
        instances = flt.env.list_instances(status="running")
        
        print(f"🏃 Running instances ({len(instances)}):")
        print("-" * 60)
        
        our_instance_found = False
        
        for instance in instances[:15]:  # Show first 15
            is_ours = test_env and instance.instance_id == test_env.instance_id
            marker = "🟢 [OURS]" if is_ours else "🔵"
            
            print(f"{marker} {instance.instance_id}")
            print(f"   🏷️  Environment: {instance.env_key}")
            print(f"   ⚡ Status: {instance.status}")
            print(f"   🌐 Subdomain: {instance.subdomain}")
            print(f"   📍 Region: {instance.region or 'N/A'}")
            print(f"   📅 Created: {instance.created_at}")
            print()
            
            if is_ours:
                our_instance_found = True
        
        if len(instances) > 15:
            print(f"... and {len(instances) - 15} more instances")
            
        if test_env:
            if our_instance_found:
                print(f"✅ Our instance {test_env.instance_id} found in running list!")
            else:
                print(f"⏳ Our instance {test_env.instance_id} not yet in running list (may still be starting)")
        
        return instances
        
    except Exception as e:
        print(f"❌ Error listing instances: {e}")
        import traceback
        traceback.print_exc()
        return []

# List running instances
running_instances = list_running_instances()


📋 Listing all running instances...
🏃 Running instances (63):
------------------------------------------------------------
🔵 5b9fc55c
   🏷️  Environment: fira
   ⚡ Status: running
❌ Error listing instances: 'EnvironmentInstance' object has no attribute 'subdomain'


Traceback (most recent call last):
  File "/var/folders/gb/s4mkbx1j5x187xr7c5rldcrw0000gn/T/ipykernel_91425/631420490.py", line 23, in list_running_instances
    print(f"   🌐 Subdomain: {instance.subdomain}")
AttributeError: 'EnvironmentInstance' object has no attribute 'subdomain'


In [ ]:
# Cleanup: Close the test environment

def cleanup_test_environment():
    """Clean up the test environment instance."""
    if not test_env:
        print("ℹ️  No test environment to clean up")
        return
        
    try:
        print(f"🧹 Cleaning up test environment: {test_env.instance_id}")
        
        # Close the environment (this will delete the instance)
        test_env.close()
        
        print(f"✅ Environment {test_env.instance_id} cleaned up successfully!")
        print("🗑️  Instance has been deleted from Fleet")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
        print("⚠️  You may need to manually delete the instance from the Fleet dashboard")
        import traceback
        traceback.print_exc()

# Cleanup (run this when you're done testing)
# cleanup_test_environment()

print("🎯 Test Complete!")
print("=" * 60)
print("✅ Package installation: SUCCESS")
print("✅ Environment creation: " + ("SUCCESS" if test_env else "FAILED"))
print("✅ Instance listing: " + ("SUCCESS" if 'running_instances' in locals() else "FAILED"))
print("✅ Manager API: " + ("SUCCESS" if test_env else "FAILED"))
print()
print("💡 Uncomment the cleanup line above to delete the test instance when done.")
